In [1]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import zarr as zr
import django
from dask.distributed import Client, progress

django.setup()
from elements.models import Zarr
from dask.distributed import Client, progress
c = Client()
import xarray, h5py

Debugging build
Hallo


In [2]:

from metamorphers.models import Metamorphing

In [3]:
met = Metamorphing.objects.last()

In [4]:
array = met.representation.loadArray()

print(array.dtype)
array = array[:, :, :, :, 0]

/code/media/zarr/sample-240
float16


In [5]:
it = array.sel(channels="Ch1-T2").astype('uint16')

In [6]:
it.dtype

dtype('uint16')

In [7]:

from cloudvolume import CloudVolume

In [8]:
info = CloudVolume.create_new_info(
	num_channels = 1,
	layer_type = 'image', # 'image' or 'segmentation'
	data_type = 'uint16', # can pick any popular uint
	encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
	resolution = [ 4, 4, 4 ], # X,Y,Z values in nanometers
	voxel_offset = [ 0, 0, 1 ], # values X,Y,Z values in voxels
	chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
	volume_size = it.shape, # X,Y,Z size in voxels
)

In [9]:
vol = CloudVolume('file:///cloud/test', info=info)
vol.provenance.description = "Test"
vol.provenance.owners = ['email_address_for_uploader/imager'] # list of contact email addresses


In [10]:
vol.commit_info() # generates gs://bucket/dataset/layer/info json file
vol.commit_provenance() # generates gs://bucket/dataset/layer/provenance json file

In [ ]:
vol[:,:,:] = it.values

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]





Uploading:   0%|          | 0/1 [00:00<?, ?it/s]







Uploading:   0%|          | 0/1 [00:00<?, ?it/s]








Uploading:   0%|          | 0/1 [00:00<?, ?it/s]









Uploading:   0%|          | 0/1 [00:00<?, ?it/s]











Uploading:   0%|          | 0/1 [00:00<?, ?it/s]












Uploading:  18%|█▊        | 20/112 [00:00<00:00, 163.97it/s]













Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  8.38it/s]

In [ ]:
import napari

with napari.gui_qt():
    # specify contrast_limits and is_pyramid=False with big data
    # to avoid unecessary computations
    napari.view_image(it, contrast_limits=[0,2000], is_pyramid=False)

In [7]:
%gui qt

tornado.application - ERROR - Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x7f26e4fa1950>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 314, in advance_eventloop
    eventloop(self)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/eventloops.py", line 129, in loop_qt5
    return loop_qt4(kernel)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/eventloops.py", line 116, in loop_qt4
    kernel.app = get_app_qt4([" "])
  File "/opt/conda/lib/python3.7/site-packages/IPython/lib/guisupport.py", line 114, in get_app_qt4
    from IPython.external.qt_for_kernel import QtGui
  File "/opt/conda/lib/python3.7/site-packages/IPython/external/qt_for_kernel.py", line 95, in <module>
    QtCore, QtGui, QtSvg, QT_API = load_qt(api_opts)
  File "/opt

In [ ]:
import napari

with napari.gui_qt():
    # specify contrast_limits and is_pyramid=False with big data
    # to avoid unecessary computations
    napari.view_image(it, contrast_limits=[0,2000], is_pyramid=False)

/opt/conda/lib/python3.7/site-packages/qtpy/__init__.py:216: RuntimeWarning: Selected binding "pyqt5" could not be found, using "pyside2"
  'using "{}"'.format(initial_api, API), RuntimeWarning)


In [6]:
cmax = array.max()
cmin = array.min()
high = 255
low = 0

In [7]:
cscale = cmax - cmin
scale = float(high - low) / cscale

In [8]:
scaled = (array - cmin) * scale + low
final = (scaled.clip(low, high) + 0.5)

In [9]:
emptychannel = xarray.DataArray((da.zeros((1256,1256,1,28))), dims=array.dims, coords={"channels":["empty",]})
end = xarray.concat([array,emptychannel],dim="channels")
end.attrs = array.attrs

In [12]:
shape_3d = end.shape[0:3]

In [14]:
rgb_dtype = np.dtype([('R', 'u1'), ('G', 'u1'), ('B', 'u1')])

In [ ]:
contugious = np.ascontiguousarray(end, dtype='u1')